In [5]:
#part 1
with open('input.txt','r') as f:
    inputs=[line.strip() for line in f.readlines()]

def largest_rectangle_area(red_tiles):
    points = [tuple(map(int, i.split(','))) for i in red_tiles]
    max_area = 0
    best_pair = None
    n = len(points)
    for i in range(n):
        x1, y1 = points[i]
        for j in range(i + 1, n):
            x2, y2 = points[j]
            if x1 != x2 and y1 != y2:
                area = (abs(x1 - x2)+1) * (abs(y1 - y2)+1)
                if area > max_area:
                    max_area = area
                    best_pair = ((x1, y1), (x2, y2))
    return max_area, best_pair

largest_rectangle_area(inputs)


(4777824480, ((85762, 83462), (16156, 14823)))

In [ ]:
#part 2 not working
def solve(points):
    red_tiles=[tuple(map(int, i.split(','))) for i in points]
    # -----------------------------------
    # 1. Build full boundary path (red+green)
    path = []
    n = len(red_tiles)

    for i in range(n):
        x1, y1 = red_tiles[i]
        x2, y2 = red_tiles[(i + 1) % n]

        dx = (x2 > x1) - (x2 < x1)
        dy = (y2 > y1) - (y2 < y1)

        x, y = x1, y1
        path.append((x, y))

        while (x, y) != (x2, y2):
            x += dx
            y += dy
            path.append((x, y))

    boundary = set(path)

    # -----------------------------------
    # 2. Coordinate Compression
    xs = sorted({x for x, _ in boundary})
    ys = sorted({y for _, y in boundary})

    x_index = {x: i for i, x in enumerate(xs)}
    y_index = {y: i for i, y in enumerate(ys)}

    W = len(xs)
    H = len(ys)

    # -----------------------------------
    # 3. Robust point-in-polygon fill
    edges = []
    for i in range(len(path)):
        x1, y1 = path[i]
        x2, y2 = path[(i + 1) % len(path)]
        edges.append((x1, y1, x2, y2))

    def point_in_poly(px, py):
        inside = False
        for x1, y1, x2, y2 in edges:
            if (y1 > py) != (y2 > py):
                xin = (x2 - x1) * (py - y1) / (y2 - y1 + 0.0) + x1
                if px < xin:
                    inside = not inside
        return inside

    grid = [[0] * W for _ in range(H)]

    for y in ys:
        for x in xs:
            ix = x_index[x]
            iy = y_index[y]
            if (x, y) in boundary:
                grid[iy][ix] = 1
            elif point_in_poly(x + 0.5, y + 0.5):
                grid[iy][ix] = 1

    # -----------------------------------
    # 4. Prefix Sum
    ps = [[0] * (W + 1) for _ in range(H + 1)]

    for y in range(H):
        for x in range(W):
            ps[y + 1][x + 1] = (
                grid[y][x]
                + ps[y][x + 1]
                + ps[y + 1][x]
                - ps[y][x]
            )

    def rect_sum(x1, y1, x2, y2):
        return (
            ps[y2 + 1][x2 + 1]
            - ps[y1][x2 + 1]
            - ps[y2 + 1][x1]
            + ps[y1][x1]
        )

    reds = [(x_index[x], y_index[y]) for x, y in red_tiles]

    # -----------------------------------
    # 5. Test only red diagonal pairs
    max_area = 0
    best = None

    for i in range(len(reds)):
        x1, y1 = reds[i]
        for j in range(i + 1, len(reds)):
            x2, y2 = reds[j]

            if x1 == x2 or y1 == y2:
                continue

            xa, xb = sorted([x1, x2])
            ya, yb = sorted([y1, y2])

            width = xb - xa + 1
            height = yb - ya + 1
            area = width * height

            if area <= max_area:
                continue

            if rect_sum(xa, ya, xb, yb) == area:
                max_area = area
                best = (
                    (xs[xa], ys[ya]),
                    (xs[xb], ys[yb])
                )

    return max_area, best


with open('input.txt','r') as f:
    inputs=[line.strip() for line in f.readlines()]

# solve(inputs) #doesnt work for larger rectanges due to memory issues


In [ ]:
#part 2
def build_lines(red_tiles):
    lines = {}
    coords = red_tiles[:]
    coords.append(coords[0])
    for i in range(1, len(coords)):
        x1, y1 = coords[i-1]
        x2, y2 = coords[i]
        if x1 != x2:
            if x1 > x2:
                x1, x2 = x2, x1
            for x in range(x1, x2 + 1):
                if x not in lines:
                    lines[x] = set()
                lines[x].add(y1)
        else:
            if y1 > y2:
                y1, y2 = y2, y1
            for y in range(y1, y2 + 1):
                if x1 not in lines:
                    lines[x1] = set()
                lines[x1].add(y)
    return lines

def max_rectangle_area(red_tiles):
    lines = build_lines(red_tiles)
    rectangles = []

    for i, (x1, y1) in enumerate(red_tiles):
        for j, (x2, y2) in enumerate(red_tiles):
            if i >= j or x1 == x2 or y1 == y2:
                continue
            xa, xb = sorted([x1, x2])
            ya, yb = sorted([y1, y2])
            area = (xb - xa + 1) * (yb - ya + 1)
            rectangles.append((area, (xa, ya), (xb, yb)))
    rectangles.sort(reverse=True)
    max_area = 0
    best_rect = None
    for area, a, b in rectangles:
        x1, y1 = a[0] + 1, a[1] + 1
        x2, y2 = b[0] - 1, b[1] - 1
        if x2 < x1 or y2 < y1:
            continue  # invalid
        is_valid = True
        for x in range(x1, x2 + 1):
            if x in lines and (y1 in lines[x] or y2 in lines[x]):
                is_valid = False
                break
        if is_valid:
            for y in range(y1, y2 + 1):
                if x1 in lines and (y in lines[x1] or y in lines[x2]):
                    is_valid = False
                    break
        if is_valid:
            max_area = area
            best_rect = (a, b)
            break
    return max_area, best_rect

with open('input.txt','r') as f:
    inputs=[line.strip() for line in f.readlines()]

red_tiles=[tuple(map(int, i.split(','))) for i in inputs]
area, pair = max_rectangle_area(red_tiles)
print("max area:", area)
print("corners:", pair)


max area: 1542119040
corners: ((5942, 50248), (94645, 67632))
